In [2]:
import cv2
import glob
import os

In [3]:
test_src = 'rawTest path'
train_src = 'rawTrain path'
test_dir = 'Extracted Test data path'
train_dir = 'Extracted Train data path'

In [4]:
def find_numbers(image_path):
    """
    Function to find contours related to digits.
    Arguments:
        image_path {str} --
    Returns:
        res {list} -- list of correct contours
    """
    res = []
#     data set images size
    IMG_SIZE = (25, 25)
    
    image_org = cv2.imread(image_path)
    image = cv2.cvtColor(image_org, cv2.COLOR_BGR2GRAY)
#     denoising images
    image = cv2.GaussianBlur(image, (3, 3), 0)
    cv2.resize(image, (1500, 1500))
    ret, thresh = cv2.threshold(image, 125, 255, cv2.THRESH_BINARY)
    dilate = cv2.dilate(thresh, None, iterations = 2)
    contours, _ = cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    failed = 0
    for contour in contours:
        if cv2.contourArea(contour) < 100:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        x_ = int(w/4)
        y_ = int(h/4)
        x -= x_; y -= y_
        w += x_; h += y_
        data = image[y:y+h, x:x+w]
#         cv2.rectangle(image_org, (x, y), (x+w, y+h), (0, 255 ,0), 1)
        try:
            data = cv2.resize(data, IMG_SIZE)
        except:
#           Wrong contours
            failed += 1
        res.append(data)
#     cv2.imwrite('Desktop/digit_clustering/{}'.format(image_path), image_org)
    print('{} numbers were found out of {} contours.'.format(len(contours)-failed, len(contours)))
    return res

In [5]:
def build_dataSet(src, dst, inputformat, outputformat):
    SEP = '/'
    files = glob.glob(os.path.join(src, '*.{}'.format(inputformat)))
    for i, file in enumerate(files):
        print(file)
        images = find_numbers(file)

        folder_name = file.split(SEP)[-1]
        path = os.path.join(dst, folder_name.split(SEP)[-1][-5])
        os.mkdir(path)
        for c, image in enumerate(images):
            try:
                cv2.imwrite(path + '{}{}.{}'.format(SEP, c, outputformat), image)
            except:
                pass

In [8]:
inputformat = 'jpg'
outputformat = 'png'

build_dataSet(test_src, test_dir, inputformat, outputformat)
build_dataSet(train_src, train_dir, inputformat, outputformat)